In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import folium
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('2001-2022storms.csv')

In [3]:
df['LAT'] = pd.to_numeric(df['LAT'], errors='coerce')
df['LONG'] = pd.to_numeric(df['LONG'], errors='coerce')

In [4]:
def create_sequences(data, window_size):
    sequences = []
    for i in range(len(data) - window_size + 1):  
        sequences.append(data[i:(i + window_size)])
    return sequences

window_size = 25
lat_sequences = create_sequences(df['LAT'].values, window_size)
long_sequences = create_sequences(df['LONG'].values, window_size)

In [5]:
sequences = list(zip(lat_sequences, long_sequences))
train_sequences, test_sequences = train_test_split(sequences, test_size=0.2, random_state=42)

In [6]:
X_train = [np.array(seq[0][:-1]).reshape(-1, 1) for seq in train_sequences]
y_train = [seq[0][-1] for seq in train_sequences]

X_test = [np.array(seq[0][:-1]).reshape(-1, 1) for seq in test_sequences]
y_test = [seq[0][-1] for seq in test_sequences]

X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [7]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(window_size, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [8]:
# Get the most recent sequence of latitude values
recent_lat_sequence = df['LAT'].values[-window_size:]

In [9]:
# Reshape the sequence to match the input shape of the model
recent_lat_sequence = recent_lat_sequence.reshape((1, window_size, 1))

# Predict the next latitude
next_lat = model.predict(recent_lat_sequence)

print(next_lat)

1/1 [==============================] - 0s 398ms/step
[[-5.4748507]]


In [10]:
#now to train the long model
X_train_long = [np.array(seq[1][:-1]).reshape(-1, 1) for seq in train_sequences]
y_train_long = [seq[1][-1] for seq in train_sequences]

X_test_long = [np.array(seq[1][:-1]).reshape(-1, 1) for seq in test_sequences]
y_test_long = [seq[1][-1] for seq in test_sequences]

X_train_long = np.array(X_train_long)
y_train_long = np.array(y_train_long)

X_test_long = np.array(X_test_long)
y_test_long = np.array(y_test_long)

In [11]:
model_long = Sequential()
model_long.add(LSTM(50, activation='relu', input_shape=(window_size, 1)))
model_long.add(Dense(1))
model_long.compile(optimizer='adam', loss='mse')


In [12]:
# Get the most recent sequence of longitude values
recent_long_sequence = df['LONG'].values[-window_size:]

In [13]:
# Reshape the sequence to match the input shape of the model
recent_long_sequence = recent_long_sequence.reshape((1, window_size, 1))

# Predict the next longitude
next_long = model_long.predict(recent_long_sequence)

print(next_long)

1/1 [==============================] - 0s 242ms/step
[[80.667404]]


In [14]:
# Create a folium map centered at the provided coordinates
map_center = [next_lat, next_long]
my_map = folium.Map(location=map_center, zoom_start=10)

# Add a marker at the specified latitude and longitude
folium.Marker(location=[next_lat, next_long], popup='Your Location').add_to(my_map)

# my_map.save('map_with_marker.html')
my_map